<a href="https://colab.research.google.com/github/filippozuddas/ML-SRT-SETI/blob/main/setigen_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install setigen blimpy

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import setigen as stg
from pathlib import Path
from astropy import units as u
from astropy.time import Time
import scipy
import os

In [10]:
BASE_DIR = "/content/filippo"
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
NOTEBOOKS_DIR = os.path.join(BASE_DIR, "notebooks")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
TEST_DIR = os.path.join(BASE_DIR, "test")
TEST_IMAGES_DIR = os.path.join(TEST_DIR, "images")

In [14]:
# Parametri Frame
fchans = 512
tchans = 16
dt = 18.253611008*u.s
df = 2.7939677238464355*u.Hz
fch1 = 1100*u.MHz

c = stg.OrderedCadence(order = 'ABABAB')

# Calcola i tempi di inizio (simili a quelli nel notebook di esempio)
mjd_start = 56789
obs_length = (tchans * dt).to(u.s).value
slew_time = 15  # Tempo di spostamento tra le osservazioni

t_start_arr = [Time(mjd_start, format='mjd').unix]
for i in range(1, 6):
    t_start_arr.append(t_start_arr[i - 1] + obs_length + slew_time)

noise_mean = 5e6

# Crea e aggiunge frame alla cadenza
for i in range(6):
    frame = stg.Frame(fchans=fchans,
                      tchans=tchans,
                      df=df,
                      dt=dt,
                      fch1=fch1,
                      t_start=t_start_arr[i])

    # Aggiungi rumore gaussiano
    frame.add_noise(x_mean=noise_mean, x_std=noise_mean/10, noise_type='gaussian')

    c.append(frame)

# Parametri del segnale ET
f_start_ET = c[0].get_frequency(index=fchans // 3)
drift_rate_ET = 0.5 * u.Hz / u.s
snr_ET = 25

# Seleziona solo i frame 'A' (ON) e aggiunge il segnale ET
c.by_label("A").add_signal(
    stg.constant_path(f_start=f_start_ET, drift_rate=drift_rate_ET),
    stg.constant_t_profile(level=c[0].get_intensity(snr=snr_ET)),
    stg.gaussian_f_profile(width=3 * abs(df.value) * u.Hz),
    stg.constant_bp_profile(level=1)
)

# Aggiunge RFI (interferenza) a tutti i frame
f_start_RFI = c[0].get_frequency(index=fchans // 2)
snr_RFI = 40

c.add_signal(
    stg.constant_path(f_start=f_start_RFI, drift_rate=0.2 * u.Hz / u.s), # RFI stazionario
    stg.constant_t_profile(level=c[0].get_intensity(snr=snr_RFI)),
    stg.sinc2_f_profile(width=2 * abs(df.value) * u.Hz),
    stg.constant_bp_profile(level=1)
)

fig = plt.figure(figsize=(10, 10))
c.plot()
fig.savefig("filippo/cadence_example.png", bbox_inches='tight')


In [11]:
FCHANS = 4096
TCHANS = 16
N_FRAMES = 6

df = 2.7939677238464355 * u.Hz
dt = 18.25361108 * u.s
fch1 = 6095.214842353016 * u.MHz

RESIZE_FACTOR = 8
FCHANS_FINAL = FCHANS // RESIZE_FACTOR

NOISE_MEAN = 58348559

